In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import joblib


df = pd.read_csv("data.csv", sep=";", encoding="latin1", decimal=",")
df.head()





,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,...,BT,Saturation,KTAS_RN,Diagnosis in ED,Disposition,KTAS_expert,Error_group,Length of stay_min,KTAS duration_min,mistriage
0,2,2,71,3,3,2,right ocular pain,1,1,2,...,36.6,100,2,Corneal abrasion,1,4,2,86,5.00,1
1,1,1,56,12,3,2,right forearm burn,1,1,2,...,36.5,NaN,4,"Burn of hand, firts degree dorsum",1,5,4,64,3.95,1
2,2,1,68,8,2,2,"arm pain, Lt",1,1,2,...,36.6,98,4,"Fracture of surgical neck of humerus, closed",2,5,4,862,1.00,1
3,1,2,71,8,1,1,ascites tapping,1,1,3,...,36.5,NaN,4,Alcoholic liver cirrhosis with ascites,1,5,6,108,9.83,1
4,1,2,58,4,3,1,"distension, abd",1,1,3,...,36.5,NaN,4,Ascites,1,5,8,109,6.60,1


In [2]:
df = df.drop(columns=["Chief_complain", "Diagnosis in ED"], errors="ignore")

In [3]:

def ktas_to_class(ktas_value):
    if ktas_value in [1, 2]:
        return "Kırmızı"
    elif ktas_value == 3:
        return "Sarı"
    elif ktas_value in [4, 5]:
        return "Yeşil"
    else:
        return "Bilinmiyor"

df["ktas_class"] = df["KTAS_RN"].apply(ktas_to_class)

In [4]:

for col in ["SBP", "DBP", "HR", "RR", "BT", "Saturation"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")


def calculate_risk(row):
    risk = 0
    if row["BT"] >= 38: risk += 1
    if row["HR"] >= 100: risk += 1
    if row["Age"] >= 70: risk += 1
    return risk

df["risk_score"] = df.apply(calculate_risk, axis=1)

In [5]:

df_encoded = pd.get_dummies(df, columns=["Sex", "Arrival mode", "Injury", "Mental", "Pain", "NRS_pain"])


label_encoder = LabelEncoder()
df_encoded["target"] = label_encoder.fit_transform(df["ktas_class"])

In [6]:

X = df_encoded.drop(columns=["ktas_class", "KTAS_RN", "target"])
y = df_encoded["target"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
drop_cols = ["ktas_class", "KTAS_RN", "target", "Saturation", "KTAS duration_min"]
X = df_encoded.drop(columns=drop_cols)
y = df_encoded["target"]

In [8]:
print("Object (kategorik) tipinde kalan sütunlar:\n", df_encoded.select_dtypes(include='object').columns.tolist())

Object (kategorik) tipinde kalan sütunlar:
 ['ktas_class']


In [9]:

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [10]:

joblib.dump(model, "model.joblib")
joblib.dump(label_encoder, "label_encoder.joblib")
joblib.dump(X.columns.tolist(), "feature_columns.joblib")

print("✅ Model başarıyla kaydedildi.")

✅ Model başarıyla kaydedildi.


In [11]:
from sklearn.metrics import accuracy_score, classification_report


y_pred = model.predict(X_test)


acc = accuracy_score(y_test, y_pred)
print(f"📊 Doğruluk (Accuracy): {acc:.2f}")


print("\n📋 Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


📊 Doğruluk (Accuracy): 0.98

📋 Sınıflandırma Raporu:
              precision    recall  f1-score   support

     Kırmızı       1.00      0.96      0.98        52
        Sarı       0.97      0.95      0.96        82
       Yeşil       0.97      1.00      0.98       120

    accuracy                           0.98       254
   macro avg       0.98      0.97      0.98       254
weighted avg       0.98      0.98      0.98       254

